In [1]:
import numpy as np
import pandas as pd
from sklearn.cluster import DBSCAN, OPTICS, KMeans
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.linear_model import LinearRegression
#from module import *
%matplotlib inline
def raman_plot():
    """
    Funzione che serve per settare la dimensione dell'immgine e i label degli assi per un plot di spettro Raman.
    """
    plt.figure(figsize=(16,6))
    plt.xlabel("wave number [1/cm]")
    plt.ylabel("Intensity")

# Import dei dati

In [59]:
# import dei centroidi
data = pd.read_csv("../data/processed/CLUSTERING_data_centres.csv")
data.drop(labels='Unnamed: 0',inplace=True,axis=1)

In [3]:
# import degli spettri puri
# definisco i nomi dei vari materiali usando il file che li contiene tutti
pure_material_names=[]
with open('../data/raw/Database Raman/BANK_LIST.dat') as f:
    pure_material_names=[i[1:len(i)-5] for i in f.readlines()]
l=[]
# calcolo la dimensione del materiale puro con più dati
for i in range(len(pure_material_names)):
    l.append(pd.read_csv('../data/raw/Database Raman/'+pure_material_names[i]+'.txt', delim_whitespace=True, names=[pure_material_names[i]+'_wl',pure_material_names[i]+'_I']).size)
max_size=int(max(l)/2)
# genero un dataframe vuoto per poter usare il metodo join
pure_materials = pd.DataFrame(np.zeros(max_size),columns=['empty'])
# importiamo i dati: nome_I (intensità) e nome_wn
for i in range(len(pure_material_names)):
    pure_materials=pure_materials.join(pd.read_csv('../data/raw/Database Raman/'+pure_material_names[i]+'.txt', delim_whitespace=True, names=[pure_material_names[i]+'_wn',pure_material_names[i]+'_I']))
    
pure_materials.drop('empty', axis = 1,inplace=True)

In [14]:
#import dei pesi dei cluster
labels=np.loadtxt("../data/processed/CLUSTERING_weights.txt")

# Interpolazione

Ora devo interpolare gli spettri puri ai dati

In [4]:
pure_materials_interpoled=pd.DataFrame(data.wn.copy())
for temp in pure_material_names:
    pure_materials_interpoled=pure_materials_interpoled.join(pd.DataFrame(np.interp(data.wn, pure_materials[temp+'_wn'] ,pure_materials[temp+'_I']),columns=[temp]))


In [5]:
#Normalizzazione
for i in pure_material_names:
    pure_materials_interpoled[i]=pure_materials_interpoled[i]/np.trapz(abs(pure_materials_interpoled[i].dropna()), x=pure_materials_interpoled[i])

# Fit

In [6]:
ols = LinearRegression(positive=True) #definisco il regressore

In [62]:
N_cluster=len(data.columns)-1
coeff=[]
for i in range(N_cluster):
    ols.fit(pure_materials_interpoled[pure_material_names], data[str(i)])#ottimizziamo il modello (lineare) su i dati di training
    coeff.append(ols.coef_)

Ora, tenendo conto dei pesi dei clustering, sommo i coeficienti pesandoli e poi li normalizzo, in modo da trovare le percentuali.

In [64]:
#numero di spettri per cluster in ordine
weights=[np.count_nonzero(labels==i) for i in range(len(data.columns)-1)] 
#moltiplico i coeficienti del cluster i-esimo per questo numero
abbundances_notnormalized=[coeff[i]*weights[i] for i in range(len(data.columns)-1)]
#e infine ho la media pesata dei coeficienti
abbundances=sum(abbundances_notnormalized)/(sum(abbundances_notnormalized).sum())
#Creo un Pandas dataframe con nomi e abbondanze
abbundance_table=pd.DataFrame({'names':pure_material_names,'abbundances':abbundances})
#riordino in base alla concenrazione
abbundance_table.sort_values('abbundances',ascending=False,inplace=True)
abbundance_table.head(5)

,names,abbundances
42,Montmorillonite,0.933737
0,Albite,0.032279
44,Nontronite,0.016749
46,Phlogopite,0.006659
29,Hedenbergite,0.001836
